In [1]:
from datetime import datetime
import multiprocessing
import numpy as np
import random
from libGenerator import setProperties
from vSystem import F
from analyseGrammar import branching_turtle_to_coords
from computeVoxel import process_network
from utils import bezier_interpolation
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import scipy.io as scio
%matplotlib widget

In [2]:
def process_vessel(args):
    d0mean, d0std, tissueVolume = args
    
    properties = {
        "k": random.uniform(1, 50),
        "epsilon": random.uniform(1, 50),
        "randmarg": random.uniform(1, 50),
        "sigma": random.uniform(1, 50),
        "stochparams": True,
    }
    
    try:
        """ Randomly assign base diameter (no dimension) """
        d0 = np.random.normal(d0mean, d0std)

        """ Set L-System properties """
        setProperties(properties)

        """ Run L-System grammar for n iterations """
        niter = random.randint(6, 14)
        turtle_program = F(niter, d0)

        """ Convert grammar into coordinates """
        coords = branching_turtle_to_coords(turtle_program, d0)

        """ Analyze / sort coordinate data """
        update = bezier_interpolation(coords)

        """ Run 3D voxel traversal to generate binary mask of L-System network """
        vol = process_network(update, tVol=tissueVolume).astype(np.uint8)
        timestamp = datetime.now().strftime("%Y%m%dT%H%M%S")
        x,y,z = np.where(vol>0)
        properties.update({
            "d0": d0, 
            "niter": niter,
            "d0mean":20.0,
            "d0std":5.0,
            "tissueVolume":(512, 512, 512),
            "timestamp": timestamp,
        })
        scio.savemat(f"vessels/sim_{timestamp}.mat",
                 {
                     "sim": vol, 
                     "x": x,
                     "y": y, 
                     "z":z, 
                     "properties": properties,
                 })
        return True

    except SyntaxError as e:
        print(f"Leading 0 error: {e}")
        return None
    except ValueError as e:
        print(f"Division by 0 error: {e}")
        return None

In [3]:
n = 6000  # No. of networks to be created
d0mean = 20.0  # Diameter of base of tree
d0std = 5.0  # Standard deviation of base diameter
tissueVolume = (512, 512, 512)  # Specify number of pixels in the tissue volume

In [4]:
# Number of parallel processes
num_processes = multiprocessing.cpu_count()//2

# Create a pool of worker processes
pool = multiprocessing.Pool(processes=num_processes)

# Create a progress bar
pbar = tqdm(total=n)

results = []
for _ in range(n): 
    args = (d0mean, d0std, tissueVolume)
    results.append(pool.apply_async(process_vessel, (args,)))

vessels = []
properties_list = []

for result in results:
    vessel_result = result.get()
    if vessel_result is not None:
        pbar.update()

# Close the pool of worker processes
pool.close()

# Wait for all the worker processes to finish
pool.join()

pbar.close()

  0%|          | 0/6000 [00:00<?, ?it/s]

Process ForkPoolWorker-8:
Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/varun/miniconda3/envs/vsys/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/varun/miniconda3/envs/vsys/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/varun/miniconda3/envs/vsys/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/varun/miniconda3/envs/vsys/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/varun/miniconda3/envs/vsys/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/varun

KeyboardInterrupt: 

  File "/home/varun/miniconda3/envs/vsys/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/varun/miniconda3/envs/vsys/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/varun/miniconda3/envs/vsys/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/varun/miniconda3/envs/vsys/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-2-2703835c8c52>", line 30, in process_vessel
    vol = process_network(update, tVol=tissueVolume).astype(np.uint8)
  File "<ipython-input-2-2703835c8c52>", line 30, in process_vessel
    vol = process_network(update, tVol=tissueVolume).astype(np.uint8)
  File "<ipython-input-2-2703835c8c52>", line 30, in process_vessel
    vol = process_network(update, tVol=tissueVolume).astype(np.uint8)
  File "<ipython-i

In [ ]:
# image = vessels[1]

In [ ]:
# image.shape

In [ ]:
# x,y,z = np.where(image>0)

In [ ]:
# fig = plt.figure()
# ax = fig.add_subplot(projection="3d")
# ax.scatter(x,y,z,s=0.01)
# plt.show()